In [ ]:
pip install -q yahoo_fin

In [ ]:
pip install --upgrade yfinance

In [3]:
mkdir -p data results log

In [4]:
from google.colab import drive
drive.mount('/content/drive')
# ! find /content/drive/MyDrive/colab/ -type f -mtime +180 -delete -print


Mounted at /content/drive


In [5]:
import numpy as np
import pandas as pd

In [ ]:
def show_prediction(name, suffix):
  df = pd.read_csv(f'results/{name}_{suffix}_predictions.csv')
  df['predicted_return'] = ((df['predicted_max']+df['predicted_min']+df['predicted_close'])/3 - df['current_price'])/df['current_price']
  return df.sort_values('predicted_return', ascending=False, key=abs)


In [ ]:
def run_command_with_check(command):
  """Runs a command, prints stderr, and raises an exception if it fails."""
  process = subprocess.run(command, shell=True, capture_output=True, text=True, check=False)

  if process.returncode != 0:
    print("stderr:", process.stderr)  # Print stderr only on error
    raise subprocess.CalledProcessError(process.returncode, process.args, output=process.stdout, stderr=process.stderr)
  else:
    # Print or process stdout if needed
    #print("stdout:", process.stdout)
    pass

  return process.returncode


In [ ]:
import os
import subprocess
def run_model_nn_cmd(prefix, version, do_train=False, window_size=None, future_days=None, hourly=False, copy_tickers=True):
    # copy drive/MyDrive/colab/results/nnreturn.py to local folder
    tickers = f"{prefix}tickers.csv"
    run_command_with_check(f"cp  /content/drive/MyDrive/colab/nnreturn.py .")
    if copy_tickers:
      run_command_with_check(f"cp  /content/drive/MyDrive/colab/tickers/{tickers} .")
    if not do_train:
      run_command_with_check(f"cp  /content/drive/MyDrive/colab/models/{prefix}_{version}_*.pth .")
      # Run nnreturn.py in results folder and throw exception if error
      today = pd.to_datetime('today').strftime('%m%d')
      run_command_with_check(f"rename 's/{version}/{today}/' *.pth")
      cmd_arg = ""
      if window_size:
        cmd_arg += f" --window-size {window_size}"
      if future_days:
        cmd_arg += f" --future-days {future_days}"
      if hourly:
        cmd_arg += " --hourly"
      run_command_with_check(f"python3 nnreturn.py --tickers-file {prefix}tickers.csv --mode analyze {cmd_arg}")
      run_command_with_check(f"cp {prefix}*predictions.csv results")
      # Get today's date

      return show_prediction(prefix, today)
    else:
      raise Exception("Not implemented")


In [ ]:
run_model_nn_cmd('etf', '0202').head(10)

In [ ]:
# 0716 is hourly 20
run_model_nn('idx', '0716', window_size=20, future_days=8, hourly=True).head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
23,XLB,89.61,2025-07-17,2025-07-19,89.62,90.66,91.40,0.010601
31,XLV,132.58,2025-07-17,2025-07-19,132.34,134.13,134.68,0.008573
20,URA,40.29,2025-07-17,2025-07-19,39.80,40.24,39.87,-0.007942
4,EEM,49.12,2025-07-17,2025-07-19,48.63,48.96,48.71,-0.007193
9,IXP,110.10,2025-07-17,2025-07-19,109.92,111.52,110.81,0.005904
14,JXI,73.21,2025-07-17,2025-07-19,72.43,73.40,72.53,-0.005782
18,SMH,292.19,2025-07-17,2025-07-19,292.37,295.67,293.22,0.005350
13,JETS,25.00,2025-07-17,2025-07-19,24.92,25.34,25.07,0.004400
11,IWO,293.85,2025-07-17,2025-07-19,292.41,296.41,296.18,0.003914
12,IYZ,29.77,2025-07-17,2025-07-19,29.50,29.77,29.72,-0.003583


In [ ]:
# pip install  git+https://github.com/ahsank/runml#egg=runml

In [ ]:
import runml.pipeline
from runml import pipeline,findata
#from importlib import reload
# reload(runml.pipeline)

In [ ]:
# Skip some days from consecutive window to lessen test /train overlap
# findata.G_NON_OVERLAP= 0

Run following if like to save results in drive

In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    # Simplify following using apply
    def calculate_alloc(row):
        gain_low = min(row['Gain_l'], 0)
        gain_high = max(row['Gain_h'], 0)
        gain = min(row['Gain'], gain_high)
        average_gain = (gain_low+gain_high + 2*gain)/4
        if (average_gain == 0):
          return 0
        avg_accuracy = (row['Accu']+row['Accu_l']+row['Accu_h'])/3
        alloc =  avg_accuracy/stop_loss - (1-avg_accuracy)*profit_factor/abs(average_gain)
        return round(max(alloc, 0), 2)
    df['Alloc'] = df.apply(calculate_alloc, axis=1)
    return df

pd.options.display.max_columns = None

In [ ]:
import os
# findata.EPOCHS=200
def run_model(prefix, version, tickers, do_train):
  if (not do_train):
    os.system(f"cp  /content/drive/MyDrive/colab/results/{prefix}-{version}-* ./results/")
  mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(),
                         pipeline.AddMA(200), pipeline.Adj()]))
  df= pipeline.runModelCombinedVola(tickers, f"{prefix}-{version}", mod, do_train)
  # Save datafrme to prefix-version-prediction.csv
  df.to_csv(f"results/{prefix}-{version}-prediction.csv")
  return df



In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax' # Change to minmax
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BASE', 'BILL', 'BMBL', 'BYRN', 'CELH', 'CFLT', 'COHR', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ELF', 'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HCP', 'HIMS', 'HOOD', 'HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JAKK', 'JKS', 'KD', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR', 'MRVL', 'MXL','MTCH', 'MU',
           'NET', 'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PAR', 'PANW',  'PAYX', 'PD', 'PI', 'PINS',
           'RBLX', 'RKLB', 'RMBS', 'RUM', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SQ', 'SPOT',
           'SOUN', 'TEM', 'TEAM', 'TDOC', 'TMDX', 'TNDM', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
df = run_model('ipos', '2a', tickers, True)

In [ ]:
addAllocHL(df, 0.10, 1)
df[(df.Gain>0)].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'CIBR', 'DAPP', 'DIA', 'DTEC', 'DXYZ',
            'EEM', 'FDN', 'FPX',
            'ICLN', 'IGV', 'IJK', 'IJR', 'IPO',
            'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'JXI', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'URA', 'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
df1 = run_model('etf', '1b', tickers1, True)



In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

In [ ]:
df1.to_csv('results/etf_12b_predictions.csv')

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax'
tickers3 = [ 'AIRI', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS',  'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CHGG', 'CLOV', 'CRCT', 'CXM',
            'DLO', 'DM', 'EVGO', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MITK', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PLUG', 'PTON', 'PUBM',
            'REAL', 'REI', 'RIG', 'RVYL',
            'SABR', 'SFIX', 'SLDP', 'SLE', 'SMWB', 'SST', 'STEM',
            'TDW', 'TDOC', 'TRUP', 'VTEX', 'VSAT', 'YEXT']
df3 = run_model('vols', '12b', tickers3, False)

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3[(df3.Gain>0)].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DELL', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'KO', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']
df4 = run_model('mcap', '1b', tickers4, True)

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
tickers5 = ['ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CX', 'DBOEY', 'DNNGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FDRVF', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']

df5 = run_model('gstock', '2a', tickers5, False)

In [ ]:
addAllocHL(df5, 0.14)
df5[df5.Gain > 0].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.G_SCALER = 'standard'
tickers6 = ['ARGT', 'EWD', 'FLN',
           'ECH', 'EPHE', 'EWA', 'EWG', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWT', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'KWT', 'THD', 'TUR', 'VNM', 'XCEM']
df6 = run_model('etf', '12a', tickers6, False)

In [ ]:
addAllocHL(df6, 0.07, 1)
df6.sort_values('Alloc', ascending=False)

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'standard'
tickers7 = ['AAL', 'ADI', 'ALB', 'AMR','ANF', 'APO', 'AQN', 'ARCH', 'ARE', 'ASC',
            'BAH', 'BAX', 'BKR', 'BLK', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IFF', 'IP', 'IPI', 'IVZ',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW', 'ZTS']
df7 = run_model('val', '1a', tickers7, False)

In [ ]:
addAllocHL(df7, 0.10, 1)
df7[df7.Gain > 0].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
tickers8 = [ 'DBC', 'FXB', 'FXE', 'FXF', 'FXY',
            'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'USO', 'UUP', 'VTIP', 'WEAT']

df8 =  run_model('com', '12a', tickers8, True)

In [ ]:
addAllocHL(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

In [ ]:
run_model_nn_cmd('pennystock', '0630').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
51,PGY,28.37,2025-10-28,2025-11-25,28.58,40.28,29.08,0.150746
45,ONDS,6.67,2025-10-28,2025-11-25,5.26,6.89,4.99,-0.143428
68,VTEX,4.53,2025-10-28,2025-11-25,4.53,5.42,5.29,0.121413
14,BLNK,1.73,2025-10-28,2025-11-25,1.39,1.74,1.43,-0.121387
69,VSAT,39.20,2025-10-28,2025-11-25,31.55,39.35,32.56,-0.120238
43,NNOX,3.88,2025-10-28,2025-11-25,3.77,4.75,4.32,0.103093
4,AP,2.33,2025-10-28,2025-11-25,2.27,2.88,2.56,0.103004
44,NXDR,2.00,2025-10-28,2025-11-25,1.70,2.17,1.58,-0.091667
36,LAZR,2.05,2025-10-28,2025-11-25,2.06,2.45,2.20,0.091057
62,SMWB,8.99,2025-10-28,2025-11-25,8.07,10.26,11.03,0.088617


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
tickers9 = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'BCH-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'NEAR-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']
df9 = run_model('cry', '11b', tickers9, False)

In [ ]:
addAllocHL(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

In [ ]:
run_model_nn_cmd('bluechipstock', '0221').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
27,MMC,178.15,2025-10-31,2025-11-28,178.75,196.82,191.02,0.060137
11,CRM,260.41,2025-10-31,2025-11-28,256.55,294.58,270.05,0.051137
20,HD,379.59,2025-10-31,2025-11-28,372.53,416.99,400.08,0.044636
1,ACN,250.10,2025-10-31,2025-11-28,247.14,274.27,257.57,0.038225
39,UNH,341.56,2025-10-31,2025-11-28,322.39,376.19,364.94,0.037905
40,UPS,96.42,2025-10-31,2025-11-28,93.77,104.90,101.04,0.036127
31,NOW,919.28,2025-10-31,2025-11-28,919.74,943.27,990.78,0.034792
30,NVDA,202.49,2025-10-31,2025-11-28,182.82,210.31,193.64,-0.034076
17,DIS,112.62,2025-10-31,2025-11-28,110.97,120.24,117.83,0.033091
16,DHR,215.38,2025-10-31,2025-11-28,211.81,229.87,225.71,0.032888


In [ ]:
run_model_nn_cmd('country', '1112').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
0,ARGT,92.62,2025-11-12,2025-12-10,88.24,100.67,101.40,0.044807
22,TUR,33.05,2025-11-12,2025-12-10,32.44,35.89,34.92,0.041351
17,ILF,30.83,2025-11-12,2025-12-10,28.91,31.36,29.97,-0.024327
4,EPHE,23.58,2025-11-12,2025-12-10,21.90,24.15,23.22,-0.020780
20,KWT,39.64,2025-11-12,2025-12-10,36.53,40.64,39.44,-0.019425
13,EWW,68.32,2025-11-12,2025-12-10,64.99,69.08,66.98,-0.019077
21,THD,59.46,2025-11-12,2025-12-10,56.10,61.44,57.47,-0.018892
23,VNM,17.48,2025-11-12,2025-12-10,16.64,17.79,17.06,-0.018116
18,INDA,54.54,2025-11-12,2025-12-10,51.04,56.71,53.65,-0.013568
12,EWT,65.32,2025-11-12,2025-12-10,63.90,67.74,66.93,0.013319


In [ ]:
run_model_nn_cmd('global', '1107').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
39,LYB,42.33,2025-11-07,2025-12-05,41.09,48.35,49.47,0.093866
23,ESEA,57.10,2025-11-07,2025-12-05,56.12,64.93,62.59,0.072037
24,EVKIY,8.05,2025-11-07,2025-12-05,7.84,9.45,8.53,0.069151
50,POAHF,40.37,2025-11-06,2025-12-04,36.64,47.17,43.08,0.047725
63,TRYIY,12.02,2025-11-07,2025-12-05,11.86,13.84,12.00,0.045480
45,NVO,45.28,2025-11-07,2025-12-05,42.94,49.57,49.39,0.044611
1,ADDDF,182.37,2025-11-07,2025-12-05,178.28,202.08,188.93,0.040540
22,ERJ,63.88,2025-11-06,2025-12-04,60.92,73.00,65.35,0.039814
62,TKAMY,14.72,2025-11-07,2025-12-05,12.95,15.07,14.44,-0.038496
25,FANUY,16.28,2025-11-07,2025-12-05,14.86,17.09,15.13,-0.036036


In [ ]:
run_model_nn_cmd('valuestock', '0301').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
63,OLN,19.96,2025-11-12,2025-12-10,19.60,22.78,21.34,0.064128
90,TWI,7.84,2025-11-12,2025-12-10,7.63,8.65,8.57,0.056548
47,HSII,58.81,2025-11-12,2025-12-10,52.91,58.10,57.06,-0.047384
4,ANF,71.28,2025-11-12,2025-12-10,69.53,78.20,76.07,0.046577
67,PETS,2.91,2025-11-12,2025-12-10,2.95,3.15,3.03,0.045819
73,RCL,264.51,2025-11-12,2025-12-10,261.45,286.09,281.79,0.045115
77,SBLK,18.85,2025-11-12,2025-12-10,18.52,21.32,19.20,0.044032
42,FMS,23.93,2025-11-12,2025-12-10,21.68,24.18,22.78,-0.043878
79,SPTN,26.90,2025-09-22,2025-10-20,25.39,27.25,24.55,-0.043494
57,MMM,170.50,2025-11-12,2025-12-10,167.33,185.09,180.54,0.041955


In [ ]:
run_model_nn_cmd('commodity', '0312').head(5)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
8,SLV,45.96,2025-11-14,2025-12-12,44.89,52.83,46.69,0.047360
5,GLD,375.96,2025-11-14,2025-12-12,367.33,428.92,380.72,0.043524
15,WEAT,4.21,2025-11-14,2025-12-12,4.12,4.40,4.27,0.012668
13,UUP,28.11,2025-11-14,2025-12-12,27.48,29.14,28.48,0.009131
4,FXY,59.53,2025-11-14,2025-12-12,58.00,61.63,60.16,0.006719


In [ ]:
run_model_nn_cmd('crypto', '0214').head(5)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
3,ADA-USD,0.67,2025-10-21,2025-11-18,0.60,0.90,0.75,0.119403
7,FIL-USD,1.61,2025-10-21,2025-11-18,1.61,1.75,1.95,0.099379
5,DOT-USD,3.13,2025-10-21,2025-11-18,2.86,3.82,3.62,0.096912
12,AVAX-USD,20.58,2025-10-21,2025-11-18,14.94,22.87,18.43,-0.089083
10,BCH-USD,502.16,2025-10-21,2025-11-18,381.60,553.39,445.81,-0.083426


In [ ]:
run_model_nn_cmd('techstock', '0312').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
83,ZS,307.92,2025-10-22,2025-11-19,243.20,308.87,245.14,-0.136994
15,DASH,252.31,2025-10-22,2025-11-19,213.76,250.49,221.27,-0.094342
41,LYFT,20.26,2025-10-22,2025-11-19,16.44,20.88,18.25,-0.085719
11,CFLT,22.58,2025-10-22,2025-11-19,21.57,25.89,25.70,0.080012
36,INTA,39.40,2025-10-22,2025-11-19,39.53,46.05,41.77,0.077411
61,PUBM,8.42,2025-10-22,2025-11-19,8.15,9.91,9.00,0.071259
81,Z,77.37,2025-10-22,2025-11-19,74.40,80.95,93.17,0.070699
30,GTLB,48.09,2025-10-22,2025-11-19,47.13,58.83,48.37,0.069730
28,FOUR,75.39,2025-10-22,2025-11-19,70.57,85.25,85.24,0.065835
21,DT,49.56,2025-10-22,2025-11-19,48.00,56.20,53.63,0.061542


In [ ]:
run_model_nn('stock', '0123').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
204,PUBM,12.82,2025-07-03,2025-07-31,12.64,16.44,14.37,0.129745
375,PI,115.86,2025-07-03,2025-07-31,116.02,144.76,128.19,0.119080
17,INTA,48.11,2025-07-03,2025-07-31,45.99,55.07,59.27,0.110857
352,ABR,10.90,2025-07-03,2025-07-31,10.53,12.98,12.60,0.104281
144,LFMD,12.50,2025-07-03,2025-07-31,11.93,14.68,14.55,0.097600
378,RCAT,7.08,2025-07-03,2025-07-31,5.73,7.56,5.93,-0.095104
10,ARQT,13.82,2025-07-03,2025-07-31,13.37,16.73,15.27,0.094308
348,CRM,272.15,2025-07-03,2025-07-31,267.03,319.47,305.57,0.092620
283,NTGR,29.41,2025-07-03,2025-07-31,28.97,35.49,31.90,0.092146
150,POWL,217.66,2025-07-03,2025-07-31,204.27,250.88,256.42,0.089727


In [6]:
import sys
sys.path.append('/content')
#!unzip drive/MyDrive/colab/stock_prediction_new.zip
import stock_prediction.stock_prediction.stock_predictor_lib
from stock_prediction.stock_prediction.stock_predictor_lib import run_model_nn, show_prediction, StockPredictorLib, Config
colab_path = 'content/drive/MyDrive/colab'

In [ ]:
run_model_nn('buystocks', '0402', False, 500, 250).head(10)
# show_prediction('buystocks', '0508').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
101,CHGG,1.29,2025-10-14,2026-09-29,1.27,2.56,2.49,0.633075
360,QBTS,43.06,2025-10-14,2026-09-29,37.01,88.83,81.98,0.608763
102,SLP,15.26,2025-10-14,2026-09-29,14.61,29.33,29.45,0.603102
82,ETSY,71.91,2025-10-14,2026-09-29,68.53,140.91,121.81,0.535484
325,BE,114.06,2025-10-14,2026-09-29,96.53,222.55,205.95,0.534368
9,HIMS,54.02,2025-10-14,2026-09-29,45.33,101.46,101.70,0.533321
348,SH,37.23,2025-10-14,2026-09-29,36.60,65.16,69.38,0.532277
295,CRNC,13.07,2025-10-14,2026-09-29,12.71,23.51,23.62,0.526141
18,OUST,36.16,2025-10-14,2026-09-29,32.69,69.77,62.96,0.524889
253,SLDP,6.94,2025-10-14,2026-09-29,6.73,13.27,11.71,0.523055


In [ ]:
techcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
colab_path = 'content/drive/MyDrive/colab'
techcfg.model_path = f'/{colab_path}/models/transformer/techstock_1005'
techdf = run_model_nn(f'/{colab_path}/tickers/techstocktickers.csv', cfg=techcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/techstock_1005_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/techstock_1005_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/techstock_1005_close_model.pth'}


In [ ]:
techdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
51,NVCR,13.86,2025-10-22,2025-11-19,13.50,18.26,19.08,0.222703
7,BILL,50.71,2025-10-22,2025-11-19,49.05,69.35,57.36,0.155328
55,PAR,37.22,2025-10-22,2025-11-19,25.63,38.41,31.02,-0.148666
8,BMBL,5.67,2025-10-22,2025-11-19,5.22,6.55,7.72,0.145797
32,HIMS,47.46,2025-10-22,2025-11-19,44.09,58.07,60.34,0.141312
11,CFLT,22.58,2025-10-22,2025-11-19,21.50,29.60,26.02,0.138471
59,PI,202.08,2025-10-22,2025-11-19,138.09,213.31,183.09,-0.118352
69,TDOC,8.34,2025-10-22,2025-11-19,6.64,9.91,5.82,-0.105915
26,ETSY,73.25,2025-10-22,2025-11-19,69.82,103.52,69.65,0.105757
79,UPWK,16.01,2025-10-22,2025-11-19,15.57,18.11,19.20,0.100979


In [ ]:
defetfcfg = Config.from_file('./stock_prediction/configs/default.yaml')
colab_path = 'content/drive/MyDrive/colab'
defetfcfg.model_path = f'/{colab_path}/models/default/etf_1008'
techdf = run_model_nn(f'/{colab_path}/tickers/etftickers.csv', cfg=defetfcfg)
techdf.head(10)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/default/etf_1008_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/default/etf_1008_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/default/etf_1008_close_model.pth'}


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
13,IYZ,33.03,2025-10-08,2025-11-05,30.87,33.49,31.85,-0.029064
6,IJR,118.99,2025-10-08,2025-11-05,108.90,122.23,116.23,-0.026921
15,JXI,79.81,2025-10-08,2025-11-05,74.77,81.14,77.11,-0.026772
20,VNQ,89.93,2025-10-08,2025-11-05,87.95,96.33,92.71,0.026687
28,XLU,90.63,2025-10-08,2025-11-05,85.39,92.23,87.12,-0.026297
30,XLY,235.95,2025-10-08,2025-11-05,230.70,251.59,242.76,0.024299
0,ARKF,58.20,2025-10-08,2025-11-05,54.67,62.53,61.62,0.024170
10,IXP,122.22,2025-10-08,2025-11-05,118.91,128.84,127.49,0.023400
23,XLE,89.17,2025-10-08,2025-11-05,81.98,92.03,88.31,-0.019401
2,ARKW,179.71,2025-10-08,2025-11-05,168.07,183.64,177.37,-0.018641


In [ ]:
tvolcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
tvolcfg.model_path = f'/{colab_path}/models/transformer/pennystock_1009'
tvoldf = run_model_nn(f'/{colab_path}/tickers/pennystocktickers.csv', cfg=tvolcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/pennystock_1009_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/pennystock_1009_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/pennystock_1009_close_model.pth'}


In [ ]:
tvoldf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
36,LAZR,2.06,2025-10-28,2025-11-25,1.38,2.69,5.37,0.527508
48,OUST,33.10,2025-10-28,2025-11-25,21.69,34.22,23.40,-0.201309
52,PLUG,2.78,2025-10-28,2025-11-25,1.91,3.54,1.34,-0.185851
59,SFIX,4.64,2025-10-28,2025-11-25,3.10,4.80,3.53,-0.178879
3,ACHR,11.07,2025-10-28,2025-11-25,9.74,11.70,17.31,0.166817
61,SLE,1.43,2025-10-28,2025-11-25,1.37,2.16,1.43,0.156177
42,NEPH,5.63,2025-10-28,2025-11-25,4.25,5.90,4.20,-0.150385
9,AXTI,6.64,2025-10-28,2025-11-25,5.34,6.99,4.62,-0.149096
64,STEM,23.28,2025-10-28,2025-11-25,10.43,26.92,23.14,-0.133877
66,TDOC,8.34,2025-10-28,2025-11-25,7.63,11.97,8.74,0.132694


In [ ]:
bctcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
bctcfg.model_path = f'/{colab_path}/models/transformer/bluechipstock_1010'
bctdf = run_model_nn(f'/{colab_path}/tickers/bluechipstocktickers.csv', cfg=bctcfg)
bctdf.head(10)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/bluechipstock_1010_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/bluechipstock_1010_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/bluechipstock_1010_close_model.pth'}


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
37,TSLA,456.56,2025-10-31,2025-11-28,320.31,463.00,411.77,-0.127475
5,AVGO,369.63,2025-10-31,2025-11-28,291.64,371.68,319.36,-0.113817
32,ORCL,262.61,2025-10-31,2025-11-28,259.45,324.23,291.42,0.110773
29,NKE,64.59,2025-10-31,2025-11-28,61.85,74.82,73.96,0.087010
18,GOOGL,281.19,2025-10-31,2025-11-28,227.82,294.69,255.64,-0.077551
10,CDNS,338.69,2025-10-31,2025-11-28,281.46,359.87,307.80,-0.065881
12,COST,911.45,2025-10-31,2025-11-28,885.64,1057.86,958.88,0.061452
35,SBUX,80.87,2025-10-31,2025-11-28,76.19,93.48,87.18,0.058695
17,DIS,112.62,2025-10-31,2025-11-28,110.92,119.69,125.36,0.053602
33,PYPL,69.27,2025-10-31,2025-11-28,64.93,70.28,62.40,-0.049083


In [ ]:
bcrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
bcrcfg.model_path = f'/{colab_path}/models/rolling/bluechipstock_1103'
bcrdf = run_model_nn(f'/{colab_path}/tickers/bluechipstocktickers.csv', cfg=bcrcfg)
bcrdf.head(10)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/bluechipstock_1103_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/bluechipstock_1103_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/bluechipstock_1103_close_model.pth'}


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
26,META,648.35,2025-10-31,2025-11-28,633.59,724.74,688.94,0.052554
39,UNH,341.56,2025-10-31,2025-11-28,333.98,370.31,370.55,0.048952
20,HD,379.59,2025-10-31,2025-11-28,367.37,422.24,402.52,0.046858
15,DELL,162.01,2025-10-31,2025-11-28,158.74,171.94,173.21,0.036747
18,GOOGL,281.19,2025-10-31,2025-11-28,261.09,289.27,267.66,-0.030288
11,CRM,260.41,2025-10-31,2025-11-28,240.03,271.22,248.23,-0.027841
16,DHR,215.38,2025-10-31,2025-11-28,199.12,228.36,201.69,-0.026264
24,KO,68.90,2025-10-31,2025-11-28,67.16,73.01,71.76,0.025302
13,CSCO,73.11,2025-10-31,2025-11-28,71.43,77.97,75.38,0.024848
12,COST,911.45,2025-10-31,2025-11-28,867.17,943.49,858.80,-0.023731


In [ ]:
buystkcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
buystkcfg.model_path = f'/{colab_path}/models/transformer/buystocks_1115'
buystkcfg.model.window_size = 500
buystkcfg.model.future_days = 250
buystkdf = run_model_nn(f'/{colab_path}/tickers/buystockstickers.csv', cfg=buystkcfg)

In [11]:
buystkdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
130,LFMD,4.63,2025-11-14,2026-10-30,3.63,11.99,64.18,4.745140
356,LAES,4.49,2025-11-14,2026-10-30,3.21,39.63,30.40,4.437268
327,OKLO,97.57,2025-11-14,2026-10-30,82.54,984.50,406.78,4.035086
315,INOD,57.21,2025-11-14,2026-10-30,39.89,373.78,401.43,3.749170
345,NUTX,107.11,2025-11-14,2026-10-30,100.94,336.41,954.96,3.332960
95,CHGG,1.09,2025-11-14,2026-10-30,0.58,4.85,8.61,3.293578
329,RGTI,25.48,2025-11-14,2026-10-30,19.92,215.40,82.17,3.153454
330,ACHR,7.88,2025-11-14,2026-10-30,7.71,64.69,23.58,3.060068
258,WKEY,9.20,2025-11-14,2026-10-30,7.15,37.14,65.95,2.994203
352,BULL,8.63,2025-11-14,2026-10-30,4.07,57.04,40.67,2.931248


In [ ]:
stkcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
stkcfg.model_path = f'/{colab_path}/models/transformer/stock_1012'
stkdf = run_model_nn(f'stocktickers.csv', cfg=stkcfg)

In [ ]:
stkdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
325,INOD,67.20,2025-11-10,2025-12-08,58.32,100.14,97.15,0.267907
18,INVZ,1.68,2025-11-10,2025-12-08,1.59,2.43,2.33,0.259921
102,AEHR,24.20,2025-11-10,2025-12-08,15.19,24.83,15.67,-0.232920
114,APP,657.50,2025-11-10,2025-12-08,609.82,765.42,995.61,0.201952
70,TEM,73.75,2025-11-10,2025-12-08,70.43,100.76,94.26,0.199774
333,PSIX,63.81,2025-11-10,2025-12-08,41.55,74.56,39.58,-0.186700
82,NBIS,113.64,2025-11-10,2025-12-08,106.20,125.48,168.72,0.174469
202,MSTR,235.86,2025-11-10,2025-12-08,222.52,329.06,272.78,0.165041
55,IONQ,55.57,2025-11-10,2025-12-08,35.47,68.40,36.25,-0.159499
140,STRL,382.71,2025-11-10,2025-12-08,269.04,416.31,279.97,-0.159224


In [ ]:
glbcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
glbcfg.model_path = f'/{colab_path}/models/transformer/global_0907'
glbdf = run_model_nn(f'globaltickers.csv', cfg=glbcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/global_0907_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/global_0907_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/global_0907_close_model.pth'}


In [ ]:
glbdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
33,HMC,29.73,2025-11-07,2025-12-05,29.62,34.22,34.07,0.097769
3,BAK,2.44,2025-11-07,2025-12-05,2.45,2.77,2.75,0.088798
38,LIN,422.22,2025-11-07,2025-12-05,420.74,476.39,477.08,0.084908
56,SE,146.78,2025-11-07,2025-12-05,141.42,163.70,169.18,0.077122
66,VNT,37.79,2025-11-07,2025-12-05,37.83,41.91,42.27,0.076211
50,POAHF,40.37,2025-11-06,2025-12-04,40.64,44.84,44.72,0.075056
59,TEF,4.19,2025-11-07,2025-12-05,4.19,4.57,4.74,0.073986
46,NXPI,199.93,2025-11-07,2025-12-05,195.62,219.84,228.47,0.073592
45,NVO,45.17,2025-11-07,2025-12-05,43.24,50.50,50.34,0.063243
35,MELI,2090.31,2025-11-07,2025-12-05,2077.38,2341.55,2241.28,0.062077


In [ ]:
valtcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
valtcfg.model_path = f'/{colab_path}/models/transformer/valuestock_1018'
valtdf = run_model_nn(f'valuestocktickers.csv', cfg=valtcfg)


In [ ]:
valtdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
71,POWL,355.91,2025-11-12,2025-12-10,318.63,406.16,497.69,0.144934
16,CC,12.51,2025-11-12,2025-12-10,11.58,15.90,14.90,0.129230
7,ARE,53.86,2025-11-12,2025-12-10,52.99,59.89,66.73,0.111586
91,UAL,99.94,2025-11-12,2025-12-10,91.43,119.48,121.23,0.107798
14,BXC,57.66,2025-11-12,2025-12-10,55.51,69.67,63.39,0.090126
44,GLW,88.71,2025-11-12,2025-12-10,86.23,108.45,94.80,0.087739
74,RH,160.72,2025-11-12,2025-12-10,156.87,193.12,172.52,0.083686
37,F,13.45,2025-11-12,2025-12-10,12.72,13.75,10.53,-0.083024
58,MOD,147.99,2025-11-12,2025-12-10,142.12,179.25,154.26,0.071311
39,FICO,1778.19,2025-11-12,2025-12-10,1648.44,2256.72,1784.05,0.066480


In [ ]:
commtcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
commtcfg.model_path = f'/{colab_path}/models/transformer/commodity_1021'
commtdf = run_model_nn(f'/{colab_path}/tickers/commoditytickers.csv', cfg=commtcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/commodity_1021_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/commodity_1021_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/commodity_1021_close_model.pth'}


In [ ]:
commtdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
5,GLD,381.15,2025-10-21,2025-11-18,377.94,408.54,416.12,0.051729
12,USO,67.83,2025-10-21,2025-11-18,67.22,69.96,74.23,0.038921
7,PALL,128.90,2025-10-21,2025-11-18,126.66,137.55,134.23,0.030359
15,WEAT,4.04,2025-10-21,2025-11-18,3.87,4.48,4.10,0.027228
0,DBC,22.08,2025-10-21,2025-11-18,20.44,22.28,22.03,-0.022494
8,SLV,47.72,2025-10-20,2025-11-17,46.37,48.63,49.92,0.012294
13,UUP,27.90,2025-10-21,2025-11-18,27.67,28.66,28.09,0.008602
4,FXY,60.58,2025-10-21,2025-11-18,60.38,61.65,60.78,0.005888
1,FXB,128.79,2025-10-21,2025-11-18,127.56,130.69,130.12,0.005176
6,ISHG,75.58,2025-10-21,2025-11-18,74.07,76.31,75.69,-0.002955


In [ ]:
cryptcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
cryptcfg.model_path = f'/{colab_path}/models/transformer/crypto_1021'
cryptdf = run_model_nn(f'/{colab_path}/tickers/cryptotickers.csv', cfg=cryptcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/crypto_1021_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/crypto_1021_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/crypto_1021_close_model.pth'}


In [ ]:
cryptdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
8,LTC-USD,96.01,2025-10-21,2025-11-18,92.26,142.93,95.96,0.149707
2,SOL-USD,194.53,2025-10-21,2025-11-18,183.08,220.31,240.31,0.103000
6,DOGE-USD,0.20,2025-10-21,2025-11-18,0.19,0.24,0.22,0.083333
17,MATIC-USD,0.22,2025-03-24,2025-04-21,0.15,0.26,0.20,-0.075758
11,ICP-USD,3.24,2025-10-21,2025-11-18,3.12,3.92,3.32,0.065844
12,AVAX-USD,20.75,2025-10-21,2025-11-18,19.90,25.64,20.64,0.063133
0,BTC-USD,113142.62,2025-10-21,2025-11-18,111504.18,129382.97,119838.14,0.062745
10,BCH-USD,505.01,2025-10-21,2025-11-18,477.52,547.41,572.59,0.054448
13,LINK-USD,18.85,2025-10-21,2025-11-18,18.22,22.25,18.81,0.048276
1,ETH-USD,4058.20,2025-10-21,2025-11-18,3981.90,4263.35,4354.49,0.034920


In [ ]:
etftcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
etftcfg.model_path = f'/{colab_path}/models/transformer/etf_1024'
etftdf = run_model_nn(f'/{colab_path}/tickers/etftickers.csv', cfg=etftcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/etf_1024_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/etf_1024_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/etf_1024_close_model.pth'}


In [ ]:
etftdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
32,XSW,202.28,2025-10-24,2025-11-21,194.48,224.41,215.06,0.044674
0,ARKF,57.09,2025-10-24,2025-11-21,55.52,59.42,63.35,0.040988
15,JXI,81.00,2025-10-24,2025-11-21,77.28,81.16,74.89,-0.039794
6,IJR,121.14,2025-10-24,2025-11-21,119.88,132.09,125.05,0.037422
25,XLI,154.91,2025-10-24,2025-11-21,145.39,158.86,143.73,-0.036042
29,XLV,146.03,2025-10-24,2025-11-21,137.05,148.31,137.14,-0.035586
28,XLU,91.39,2025-10-24,2025-11-21,86.95,92.50,85.14,-0.034942
31,XME,99.82,2025-10-24,2025-11-21,95.85,113.94,99.56,0.033026
7,IPO,49.72,2025-10-24,2025-11-21,42.90,50.84,51.17,-0.028493
22,XLC,115.55,2025-10-24,2025-11-21,108.90,116.89,111.75,-0.026280


In [ ]:
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
import pandas as pd
from google.auth import default


auth.authenticate_user()


creds, _ = default()
gc = gspread.authorize(creds)
# gc = gspread.authorize(GoogleCredentials.get_application_default())


In [ ]:
# Open your spreadsheet by name
spreadsheet = gc.open('Allocation')
# Or by URL: spreadsheet = gc.open_by_url('Your Spreadsheet URL')

# Select the worksheet (e.g., the first sheet)
worksheet = spreadsheet.worksheet("Buy")

# Get all values as a list of lists
data = worksheet.get_all_values()

# Convert to a Pandas DataFrame
sheetdf = pd.DataFrame(data[1:], columns=data[0]) # Assuming first row is header


In [ ]:
df = sheetdf[['Ticker']].copy()

# Apply the extraction function first, then filter based on the result
def extract_ticker(ticker_str):
    # Convert to string before splitting
    ticker_str = str(ticker_str)
    parts = ticker_str.split(':')
    if len(parts) > 1:
        return parts[1].strip() # Use strip to remove leading/trailing whitespace
    return ticker_str.strip() # Return the original string if no colon, also strip whitespace

df['Ticker'] = df['Ticker'].apply(extract_ticker)
# Remove BTCUSD and add BTC-USD, ETH-USD, ADA-USD, SOL-USD in Ticker
df = df[df['Ticker'] != 'BTCUSD']
df = pd.concat([df, pd.DataFrame({'Ticker': ['BTC-USD', 'ETH-USD', 'ADA-USD', 'SOL-USD']})], ignore_index=True)
df.to_csv('stocktickers.csv', index=False)
df.to_csv('buystockstickers.csv', index=False)

In [ ]:
# Only have Tickers column

run_model_nn('buystocks', '0402', False, 500, 250, copy_tickers=False).head(10)


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
333,QBTS,34.25,2025-10-08,2026-09-23,32.44,71.01,65.57,0.644964
96,CHGG,1.42,2025-10-08,2026-09-23,1.40,2.82,2.74,0.633803
97,SLP,16.47,2025-10-08,2026-09-23,16.09,32.07,31.69,0.616070
18,OUST,30.65,2025-10-08,2026-09-23,29.45,61.59,56.80,0.607830
9,HIMS,58.17,2025-10-08,2026-09-23,49.10,110.63,110.40,0.547934
252,LYFT,21.16,2025-10-08,2026-09-23,19.66,41.40,36.93,0.543636
6,DOMO,14.74,2025-10-08,2026-09-23,12.98,29.59,25.17,0.531886
326,ROOT,81.73,2025-10-08,2026-09-23,76.40,156.09,141.76,0.526367
122,LZ,9.97,2025-10-08,2026-09-23,9.63,19.32,16.65,0.524574
276,CRNC,11.96,2025-10-08,2026-09-23,11.62,21.43,21.43,0.518395


In [ ]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +180 -delete -print

sending incremental file list
buystocks_0709_predictions.csv

sent 18,897 bytes  received 35 bytes  37,864.00 bytes/sec
total size is 18,770  speedup is 0.99
